<a href="https://colab.research.google.com/github/greapis/Python_practice/blob/main/Tensorflow_AI_LS_CIFAR_10(CNN).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

CIFAR-10 Datasets은 제공됨
 - MNIST : from tensorflow.keras.datasets import mnist
 - CIFAR-10 Datasets 역시 keras에서 기본적으로 제공

In [10]:
# Road library

import tensorflow as tf
from tensorflow.keras.datasets import cifar10
  #  keras에서 Datasets 제공

from tensorflow.keras import layers, models
from tensorflow.keras.utils import to_categorical
  # using "One-Hot Encoding"

import numpy as np
from PIL import Image

Data 전처리
 - X : 정규화(MNIST와 동일)

     각 픽셀은 0과 255 사이의 정수값으로 설정됨
     해당 값들을 0과 1로 정규화하는 것이 필요

 - Y : Label Encoding : One-Hot Encoding
     각 클래스를 독립적인 카테고리로 인식하기 위함
         airplaine  = [1,0,0,0,0,0,0,0,0,0]
         automobile = [0,1,0,0,0,0,0,0,0,0]   

In [11]:
# Data 전처리

# Roding CIFAR-10 datasets
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
  # x : pixels value(0~255),
  # y : labels (airplane, bird 등)

# Ont-Hot Encoding 적용
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

# Nomalization data
x_train, x_test = x_train / 255.0, x_test / 255.0

모델 구축 : 1번 합성곱 + 풀링 layer

 - 합성곱 layer(Convolutional Layer)

   필터수, 커널 크기(커널 = filter), 활성화 함수 설정

   input_shape = (32,32,3)인 이유는 RGB가 포함되었기 때문
   

 - Pooling Layer : Max Pooling

   Pooling filter size : (2 X 2)

   Max Pooling : (2 X 2) 블록 내에서 가장 큰 값을 선정


In [12]:
# CNN Model_1 : Convolutional + Pooling Layers

model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32,32,3)))
  #  Conv2D(32, (3, 3))) 32:필터 수, (3, 3) : 커널 크기

model.add(layers.MaxPooling2D(2,2))
  #  MaxPooling2D(2,2) : Pooling filer 크기 : (2,2)

model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D(2,2))
  #  Convolutional layer, MaxPooling layer를 2번 반복
    #  모델 성능을높이고 싶다면 'Convolutional layer, MaxPooling layer' 부분 추가 가능


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


스트라이드와 패딩 (stride // padding)


  - Stride

Stride는 필터가 입력 데이터를 얼마나 이동하는지를 나타냅니다. 예를 들어, stride가 1이면 필터는 한 번에 한 칸씩 이동하고, stride가 2이면 두 칸씩 이동합니다. Stride가 크면 입력 데이터의 크기가 더 많이 줄어듭니다.

stride가 크면, filter의 이동이 크고 feature map이 적고
stride가 작으면, filter의 이동이 작고 feature map이 크고



  - Padding

Padding은 입력 데이터의 테두리에 추가적인 픽셀을 추가하는 것을 말합니다. 패딩은 일반적으로 두 가지 이유로 사용됩니다:

출력 크기 조정: 패딩을 사용하지 않으면 합성곱 연산 후 출력 데이터의 크기가 줄어들 수 있습니다. 패딩을 사용하면 원래 입력 데이터와 동일한 크기의 출력을 유지할 수 있습니다.

경계 정보 보존: 입력 데이터의 경계 부분을 보존하고 싶을 때 패딩을 사용합니다.

In [13]:
# Example of stride and padding

# - Stride
#    strides = 1
#    strides = 2

# - Padding
#    padding = 'valid' : padding 추가 X
#    padding = 'same'  : padding 추가 : 입력크기 = 출력크기 (동일)

In [14]:
# Revising CNN Code (조정 of stride and padding)

model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), strides=1, padding ='same', activation='relu', input_shape=(32, 32, 3)))
model.add(layers.MaxPooling2D(pool_size=(2, 2), strides=2))

model.add(layers.Conv2D(64, (3, 3), strides=1, padding='same', activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2), strides=1))


모델 구축 : 2번 Fully Connected Layer (완전 연결 layers)


 - Fully Connected Layer는 CNN의 마지막 부분에 위치

   이전 층에서 추출된 특징들을 바탕으로 최종 예측을 수행합니다.


 - Flatten

   완전 연결 레이어에 데이터 1D로 변환
   

 - Softmax

   다중 클래스 분류 문제에서 각 클래스 확률을 출력하기 위함

   각 클래스 확률 합 = '1'


 - 완전 연결 레이어의 역할

    특징 결합: 이전 합성곱 층과 풀링 층에서 추출된 특징들을 결합

    예측 수행: 각 클래스에 대한 확률을 계산하여 최종 분류를 수행

 - 작동 원리
  
    입력: 이전 층에서 추출된 특징 맵(feature map)이 1차원 벡터로 변환, 이를 벡터화(flattening)라고 함

    가중치와 곱셈: 입력 벡터와 가중치(weight)가 곱해지고, 편향(bias)이 추가

    활성화 함수: ReLU(Rectified Linear Unit) 또는 소프트맥스(Softmax) 적용

In [15]:
# CNN Model_2 : Fully Connected Layers(FC)

model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
    # Dense(64, )  64는 nodes의 수
model.add(layers.Dense(10, activation='softmax'))
    # Dense(10, )  10는 nodes의 수, 10개의 nodes로 한 이유는 'CIFAR-10'의 Class가 10개 이기 때문에

    # 'Softmax' activation은 각 클래스에 대한 확률을 합이 '1'이 됨
    #  ex) 첫 번째 'airplane'의 확률 = [0.982, 0.001, 0.0003, 0.00005, 0.00005, 0.000005, 0.0054, 0.0000005, 0.0000746, 0.011115]

Model Compile

In [20]:
# Model Compile

#  환경 설정
#   Optimizer : Adam
#   Loss : Categorical Crossentropy
#   Metrics : Accuracy

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

모델 학습

 - 일련의 설정 기반으로 모델 학습

   model.fit() : 구성된 모델로 학습해라

   학습 DATA : (Image) x_train,  (Labels) y_train

   Hyper Parameter : epoch, batch_size

   Ratio of Learning/Test


 - Designating Hyper Parameter

   epoch : 10 / batch_size : 32

 - Setting Ratio of Learning/Test

   Validation_split : 검증 데이터 비율 = 20%

   학습 데이터 수 = 50,000 * 0.8 = 40,000

In [21]:
# Code : Learning Model

model.fit(x_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

Epoch 1/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 101s 80ms/step - accuracy: 0.3382 - loss: 1.7898 - val_accuracy: 0.5833 - val_loss: 1.1916
Epoch 2/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 143s 80ms/step - accuracy: 0.6074 - loss: 1.1098 - val_accuracy: 0.6540 - val_loss: 0.9922
Epoch 3/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 141s 80ms/step - accuracy: 0.6656 - loss: 0.9406 - val_accuracy: 0.6728 - val_loss: 0.9493
Epoch 4/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 98s 78ms/step - accuracy: 0.7083 - loss: 0.8327 - val_accuracy: 0.6811 - val_loss: 0.9256
Epoch 5/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 139s 76ms/step - accuracy: 0.7455 - loss: 0.7349 - val_accuracy: 0.6920 - val_loss: 0.9026
Epoch 6/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 144s 78ms/step - accuracy: 0.7710 - loss: 0.6589 - val_accuracy: 0.6961 - val_loss: 0.9106
Epoch 7/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 95s 76ms/step - accuracy: 0.7892 - loss: 0.5990 - val_accuracy: 0.6929 - val_loss: 0.9462
Epoch 8/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 144s 78ms/step - accuracy: 0.

모델 성능 평가

 - 모델 성능지표 계산



 - evaluate는 반환 순서 고정됨



 - f"문자열 {변수/표현식}"